# Sprint 11 - Proyecto

## Descripción del Proyecto

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

* Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región        seleccionada: calidad de crudo y volumen de reservas.
* Crear un modelo para predecir el volumen de reservas en pozos nuevos.
* Elegir los pozos petrolíferos que tienen los valores estimados más altos.
* Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.
  
Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

## Condiciones

* Solo se debe usar la regresión lineal para el entrenamiento del modelo.
* Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
* El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
* Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
* Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.
  
Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

## Descripción de los datos

Los datos de exploración geológica de las tres regiones se almacenan en archivos:

* /datasets/geo_data_0.csv. Descarga el conjunto de datos
* /datasets/geo_data_1.csv. Descarga el conjunto de datos
* /datasets/geo_data_2.csv. Descarga el conjunto de datos

* id — identificador único de pozo de petróleo

* f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)

* product — volumen de reservas en el pozo de petróleo (miles de barriles).

## Instrucciones del Proyecto

## 1 Descarga y prepara los datos. Explica el procedimiento.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

### 1.1 Preparación de datos primera región

In [2]:
df_0.info()
df_0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
df_0.duplicated().sum()

0

In [4]:
df_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Primer df, no tiene duplicados, ni valores ausentes y el tipo de datos esta bien por ahora

### 1.2 Preparación de datos segunda región

In [5]:
df_1.info()
df_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
df_1.duplicated().sum()

0

In [7]:
df_1.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Segundo df, no tiene duplicados, ni valores ausentes y el tipo de datos esta bien por ahora

### 1.3 Preparación de datos Tercera Región

In [8]:
df_2.info()
df_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
df_2.duplicated().sum()

0

In [10]:
df_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Segundo df, no tiene duplicados, ni valores ausentes y el tipo de datos esta bien por ahora

## 2 Entrena y prueba el modelo para cada región en geo_data_0.csv

### 2.1 Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

### 2.2 Entrena el modelo y haz predicciones para el conjunto de validación.
### 2.3 Guarda las predicciones y las respuestas correctas para el conjunto de validación.

In [11]:
def entrenar_region(df, random_state=12345):
    # Quitamos columnas no numéricas (ejemplo: 'id')
    df_numeric = df.drop(columns=['id'], errors='ignore')
    
    # Split
    train, valid = train_test_split(df_numeric, test_size=0.25, random_state=random_state)
    features_train = train.drop('product', axis=1)
    target_train = train['product']
    features_valid = valid.drop('product', axis=1)
    target_valid = valid['product']
    
    # Modelo
    model = LinearRegression()
    model.fit(features_train, target_train)
    pred_valid = model.predict(features_valid)
    
    # Métricas
    rmse = mean_squared_error(target_valid, pred_valid, squared=False)
    mean_pred = pred_valid.mean()
  
  
    return model, pred_valid, target_valid, mean_pred, rmse


### 2.4 Muestra el volumen medio de reservas predicho y RMSE del modelo.

In [12]:
### for para correr los 3 df
for i, df in enumerate([df_0, df_1, df_2]):
    model, pred, target, mean_pred, rmse = entrenar_region(df)
    print(f"--- Región {i} ---")
    print("Media de predicciones:", mean_pred)
    print("RMSE:", rmse)
    print()

--- Región 0 ---
Media de predicciones: 92.59256778438035
RMSE: 37.5794217150813

--- Región 1 ---
Media de predicciones: 68.728546895446
RMSE: 0.893099286775617

--- Región 2 ---
Media de predicciones: 94.96504596800489
RMSE: 40.02970873393434



### 2.5 Analisis de resultados

La región 1 es la más precisa: tiene un RMSE bajísimo (0.89). Eso significa que el modelo está prediciendo con casi exactitud el valor real de product.

Las regiones 0 y 2 tienen un error alto (37–40): ahí la regresión lineal no está capturando bien la relación entre las variables y el product. Puede que los datos de esas regiones sean más dispersos o no sigan una relación lineal tan clara.

## 3 Cálculo de ganancias

### 3.1 Almacena todos los valores necesarios para los cálculos en variables separadas. 

### 3. 2 Compara esta cantidad con la cantidad media de reservas en cada región.

In [13]:
precio_unidad = 4500       # USD por unidad de reserva
n_pozos_total = 500        # muestra aleatoria de pozos
n_pozos = 200    # pozos que se desarrollan
n_simulaciones = 1000      # número de simulaciones
inversion_total = 100_000_000  # inversión inicial (100 millones USD)

# Función de simulción
def simular_ganancias(df_region, n_simulaciones=1000):
    ganancias = []
    for _ in range(n_simulaciones):
        # Muestra aleatoria de 500 pozos
        muestra = df_region['product'].sample(n=n_pozos, replace=True, random_state=None)
        #  200 pozos más productivos
        seleccion = muestra.sort_values(ascending=False).head(n_pozos)
        # Calcular ingresos
        ingresos = seleccion.sum() * precio_unidad
        # ganancia neta
        ganancia = ingresos - inversion_total
        ganancias.append(ganancia)
    return ganancias

# Simulaciones para cada región
ganancias_r0 = simular_ganancias(df_0)
ganancias_r1 = simular_ganancias(df_1)
ganancias_r2 = simular_ganancias(df_2)

# Resultados principales
print("Región 0")
print("Ganancia media:", np.mean(ganancias_r0))
print("Probabilidad de pérdida:", (np.array(ganancias_r0) < 0).mean())

print("\nRegión 1")
print("Ganancia media:", np.mean(ganancias_r1))
print("Probabilidad de pérdida:", (np.array(ganancias_r1) < 0).mean())

print("\nRegión 2")
print("Ganancia media:", np.mean(ganancias_r2))
print("Probabilidad de pérdida:", (np.array(ganancias_r2) < 0).mean())

Región 0
Ganancia media: -16644423.094570404
Probabilidad de pérdida: 1.0

Región 1
Ganancia media: -38029771.31438186
Probabilidad de pérdida: 1.0

Región 2
Ganancia media: -14553986.328440167
Probabilidad de pérdida: 1.0


### 3.3 Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.

Región 0
Ganancia media: –16.6 M USD
Probabilidad de pérdida: 100%
Siempre pierdes dinero en las simulaciones.

Región 1
Ganancia media: –37.9 M USD
Probabilidad de pérdida: 100%
Es la peor región, pérdidas mucho mayores.

Región 2
Ganancia media: –14.6 M USD
Probabilidad de pérdida: 100%
La “menos mala”, pero aun así se pierden más de 14 M de media.

Todas las medias están por debajo del umbral (111.1) por lo tanto, en promedio, ninguna región logra cubrir los costos.
Eso explica que las 3 tengan probabilidad de pérdida = 100%.

## 4.1 Función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados

### 4.1 Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv')

In [14]:
# Parámetros globales
inversion_total = 100000000  # inversión total en dólares
n_pozos = 200                  # número de pozos a perforar
precio_unidad = 4500           # $ por unidad de reserva

def calcular_ganancia(predicciones, n_pozos=n_pozos):
    # Orden descendente y tomar los mejores pozos
    top_pozos = predicciones.sort_values(ascending=False).head(n_pozos)
    
    # Volumen total de reservas estimadas
    volumen_total = top_pozos.sum()
    
    # Ingresos en USD
    ingresos = volumen_total * precio_unidad
    
    # Ganancia neta
    ganancia = ingresos - inversion_total
    
    return ganancia, volumen_total, top_pozos

### 4.2 Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.

### 4.3 Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.

In [15]:
# función simple que use solo las características numéricas
def entrenar_modelo_simple(df_region):
    # Solo usar las características numéricas (f0, f1, f2)
    X = df_region[['f0', 'f1', 'f2']]
    y = df_region['product']

    model = LinearRegression()
    model.fit(X, y)
    return model

# Entrenar modelos para cada región
model_0 = entrenar_modelo_simple(df_0)
model_1 = entrenar_modelo_simple(df_1)
model_2 = entrenar_modelo_simple(df_2)

# Predicciones para cada región
pred_0 = pd.Series(model_0.predict(df_0[['f0', 'f1', 'f2']]))
pred_1 = pd.Series(model_1.predict(df_1[['f0', 'f1', 'f2']]))
pred_2 = pd.Series(model_2.predict(df_2[['f0', 'f1', 'f2']]))

# Cálculo de las ganancias
ganancia_0, volumen_0, top_0 = calcular_ganancia(pred_0)
ganancia_1, volumen_1, top_1 = calcular_ganancia(pred_1)
ganancia_2, volumen_2, top_2 = calcular_ganancia(pred_2)

print("Región 0:")
print(f" Volumen total (predicho): {volumen_0:.2f}")
print(f" Ganancia potencial: {ganancia_0:,.2f} USD\n")

print("Región 1:")
print(f" Volumen total (predicho): {volumen_1:.2f}")
print(f" Ganancia potencial: {ganancia_1:,.2f} USD\n")

print("Región 2:")
print(f" Volumen total (predicho): {volumen_2:.2f}")
print(f" Ganancia potencial: {ganancia_2:,.2f} USD\n")

Región 0:
 Volumen total (predicho): 32655.65
 Ganancia potencial: 46,950,407.90 USD

Región 1:
 Volumen total (predicho): 27832.02
 Ganancia potencial: 25,244,096.62 USD

Región 2:
 Volumen total (predicho): 31325.41
 Ganancia potencial: 40,964,343.83 USD



La región 0 es la que mayor tienen una ganancia potencial y volumen, después puede ser la región 2 y hasta el final la 1, por lo tanto la región 1 seria la ideal para el desarrollo de pozos petróliferos.

## 5 Calculo de riesgos y gancias para cada región

### 5.1 Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.

### 5.2 Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.

In [16]:
def bootstrap(region_pred, precio_por_barril=4500, n_pozos=500, n_sim=1000):
    ganancias = []
    
    for _ in range(n_sim):
        sample = region_pred.sample(n_pozos, replace=True)  # muestreo con reemplazo
        volumen = sample.sum()
        ganancia = volumen * precio_por_barril - 10_000_000  # costeo (ajusta si aplica)
        ganancias.append(ganancia)
    
    ganancias = np.array(ganancias)
    
    # métricas
    promedio = ganancias.mean()
    ic_95 = (np.percentile(ganancias, 2.5), np.percentile(ganancias, 97.5))
    riesgo = (ganancias < 0).mean() * 100  # probabilidad %
    
    return promedio, ic_95, riesgo

# Aplicación a cada región
res_0 = bootstrap(pred_0)
res_1 = bootstrap(pred_1)
res_2 = bootstrap(pred_2)

print("Región 0:", res_0)
print("Región 1:", res_1)
print("Región 2:", res_2)

Región 0: (198221537.55887324, (194011861.30371305, 202876055.97320813), 0.0)
Región 1: (144540660.91296405, (135058450.5841765, 153976655.86945733), 0.0)
Región 2: (203752729.50619513, (199925081.00473645, 207359946.05482334), 0.0)


Resultados del Bootstrap
* Región 0
* Beneficio promedio: 198.13 M USD
* IC 95%: 193.48 M ; 202.39 M
* Riesgo de pérdida: 0.0%

* Región 1
* Beneficio promedio: 144.70 M USD
* IC 95%: 136.32 M ; 153.98 M
Riesgo de pérdida: 0.0%

* Región 2
* Beneficio promedio: 203.64 M USD
* IC 95%: 199.70 M ; 207.33 M
* Riesgo de pérdida: 0.0%

### 5.3 Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

La mejor región para el desarrollo de pozos petrolíferos es la Región 2, ya que: 

* Presenta el mayor beneficio promedio esperado
* Su intervalo de confianza es completamente positivo y alto,
* El riesgo de pérdida es nulo.

Esto coincide con tu análisis anterior en el 4.3 , aunque allí Región 0 también era atractiva. Con el análisis de riesgos, queda más que claro Región 2 es la elección más sólida.

## Conclusión Final

Todas las regiones muestran un riesgo nulo de pérdidas (0.0%), por lo tanto ninguna presenta escenarios negativos.

La Región 2 obtiene el beneficio promedio más alto (203.64 M USD) y su intervalo de confianza también se sitúa por encima de las demás.

La Región 0 también es competitiva, con 198.13 M USD, aunque ligeramente por debajo de la Región 2.

La Región 1 es la menos atractiva, con un beneficio significativamente menor (144.70 M USD).

Se recomienda invertir en la Región 2, ya que maximiza los beneficios esperados con un riesgo nulo, posicionándola como la opción más sólida y rentable.